## Test the environment `HerdEnv` of `herd_env.py`

In [1]:
import numpy as np
import tensorflow as tf
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.utils import common
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential

In [2]:
from herd_env_ddpg import HerdEnv_DDPG

In [3]:
# sanity check
henv_val = HerdEnv_DDPG(herd_sizes = [64,64], rand_recovery_prob = 0.1, rand_infection_prob = 0.05)
utils.validate_py_environment(henv_val, episodes=10)

In [4]:
# create Herd Environment instance to be trained for
max_episode_length=100
num_herds=2
henv = HerdEnv_DDPG(herd_sizes = [32,32], expected_episode_length=-1, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.04, rand_infection_prob = 0.05)

In [5]:
# show interor values of environment
time_step = henv.reset()
print(time_step)
cumulative_reward = time_step.reward
finished = False

while not finished:
  time_step = henv.step([0.,0.]) # do nothing
  s = henv.get_state()
  print("state: ", s, "observation: ", time_step.observation, "\treward: ", time_step.reward)
  cumulative_reward += time_step.reward
  if time_step.step_type == StepType.LAST:
    finished = True

print('Final Reward = ', cumulative_reward)

TimeStep(
{'discount': 1.0,
 'observation': array([0., 0., 0.], dtype=float32),
 'reward': 0.0,
 'step_type': array(0, dtype=int32)})
state:  [2 1 2 1 1 1] observation:  [0.01 0.01 0.01] 	reward:  0.0
state:  [4 2 6 3 2 2] observation:  [0.02 0.02 0.02] 	reward:  0.0
state:  [ 4  2 10  5  3  3] observation:  [0.03 0.03 0.03] 	reward:  0.0
state:  [ 4  6 14 11  4  4] observation:  [0.04 0.04 0.04] 	reward:  0.0
state:  [ 9  5 23 16  5  5] observation:  [0.05 0.05 0.05] 	reward:  0.0
state:  [10  8 33 24  6  6] observation:  [0.06 0.06 0.06] 	reward:  0.0
state:  [13 11 46 35  7  7] observation:  [0.07 0.07 0.07] 	reward:  0.0
state:  [13 15 59 50  8  8] observation:  [0.08 0.08 0.08] 	reward:  0.0
state:  [15 17 74 67  9  9] observation:  [0.09 0.09 0.09] 	reward:  0.0
state:  [15 19 89 86 10 10] observation:  [0.1 0.1 0.1] 	reward:  0.0
state:  [ 15  20 104 106  11  11] observation:  [0.11 0.11 0.11] 	reward:  0.0
state:  [ 17  16 121 122  12  12] observation:  [0.12 0.12 0.12] 	reward

In [6]:
action_spec = henv.action_spec()
ts_spec = henv.time_step_spec()
print("action spec:\n", action_spec, "\n\ntime step spec:\n", ts_spec)

action spec:
 BoundedArraySpec(shape=(2,), dtype=dtype('float32'), name='action', minimum=0.0, maximum=1.0) 

time step spec:
 TimeStep(
{'discount': BoundedArraySpec(shape=(), dtype=dtype('float32'), name='discount', minimum=0.0, maximum=1.0),
 'observation': BoundedArraySpec(shape=(3,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0),
 'reward': ArraySpec(shape=(), dtype=dtype('float32'), name='reward'),
 'step_type': ArraySpec(shape=(), dtype=dtype('int32'), name='step_type')})


### Define scripted policies

In [7]:
# do nothing policy: cull never
action_script0 = [(max_episode_length, [0,0])]

# cull first herd every 20th step and second herd every 20th step
action_script1 = [(19, [0,0]), 
                 (1, [1,0]),
                 (19, [0,0]), 
                 (1, [0,1])] * int(1+max_episode_length/40)

manual_scripted_policy0 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script0)

manual_scripted_policy1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=ts_spec,
    action_spec=action_spec,
    action_script=action_script1)

init_policy_state = manual_scripted_policy0.get_initial_state()

In [8]:
policy_state =  init_policy_state
ts0 = henv.reset()
for _ in range(21):
    action_step = manual_scripted_policy1.action(ts0, policy_state)
    policy_state = action_step.state
    print("action=", action_step.action, "\tpolicy_state", policy_state)
policy_state = manual_scripted_policy1.get_initial_state()

action= [0. 0.] 	policy_state [0, 1]
action= [0. 0.] 	policy_state [0, 2]
action= [0. 0.] 	policy_state [0, 3]
action= [0. 0.] 	policy_state [0, 4]
action= [0. 0.] 	policy_state [0, 5]
action= [0. 0.] 	policy_state [0, 6]
action= [0. 0.] 	policy_state [0, 7]
action= [0. 0.] 	policy_state [0, 8]
action= [0. 0.] 	policy_state [0, 9]
action= [0. 0.] 	policy_state [0, 10]
action= [0. 0.] 	policy_state [0, 11]
action= [0. 0.] 	policy_state [0, 12]
action= [0. 0.] 	policy_state [0, 13]
action= [0. 0.] 	policy_state [0, 14]
action= [0. 0.] 	policy_state [0, 15]
action= [0. 0.] 	policy_state [0, 16]
action= [0. 0.] 	policy_state [0, 17]
action= [0. 0.] 	policy_state [0, 18]
action= [0. 0.] 	policy_state [0, 19]
action= [1. 0.] 	policy_state [1, 1]
action= [0. 0.] 	policy_state [2, 1]


### ... and a random policy

In [9]:
random_policy = random_py_policy.RandomPyPolicy(time_step_spec=ts_spec, action_spec=action_spec)

## Drive a rollout

In [10]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        policy_state = None # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action[0] > 0 or action_step.action[1] > 0:
            cullsteps += 1
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, HerdEnv_DDPG):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>4} action: ", action_step.action, "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  return avg_return, cullsteps

In [11]:
random_reward, cullsteps = compute_avg_return(henv, random_policy)
print (f"average return of random policy: {random_reward:.3f} avg steps with culls per episode: {cullsteps}")

average return of random policy: -1752.800 avg steps with culls per episode: 100.0


In [12]:
# show states for one rollout of second scripted policy
compute_avg_return(henv, manual_scripted_policy1, num_episodes=1, verbose=True)

episode  0 step   1 action:  [0. 0.] state= [1 0 1 0 1 1] obs= [0.01 0.01 0.01] reward= 0.0
episode  0 step   2 action:  [0. 0.] state= [4 0 5 0 2 2] obs= [0.02 0.02 0.02] reward= 0.0
episode  0 step   3 action:  [0. 0.] state= [ 6  3 11  3  3  3] obs= [0.03 0.03 0.03] reward= 0.0
episode  0 step   4 action:  [0. 0.] state= [ 7  4 18  7  4  4] obs= [0.04 0.04 0.04] reward= 0.0
episode  0 step   5 action:  [0. 0.] state= [10  6 28 13  5  5] obs= [0.05 0.05 0.05] reward= 0.0
episode  0 step   6 action:  [0. 0.] state= [11  7 39 20  6  6] obs= [0.06 0.06 0.06] reward= 0.0
episode  0 step   7 action:  [0. 0.] state= [11  9 50 29  7  7] obs= [0.07 0.07 0.07] reward= 0.0
episode  0 step   8 action:  [0. 0.] state= [13 13 63 42  8  8] obs= [0.08 0.08 0.08] reward= 0.0
episode  0 step   9 action:  [0. 0.] state= [13 13 76 55  9  9] obs= [0.09 0.09 0.09] reward= 0.0
episode  0 step  10 action:  [0. 0.] state= [14 14 90 69 10 10] obs= [0.1 0.1 0.1] reward= 0.0
episode  0 step  11 action:  [0. 0.

(-1534.5, 5.0)

In [13]:
manual_reward0, cullsteps = compute_avg_return(henv, manual_scripted_policy0, num_episodes=500)
print (f"average return of do-nothing-policy: {manual_reward0:.3f} avg culls {cullsteps}")
manual_reward1, cullsteps = compute_avg_return(henv, manual_scripted_policy1, num_episodes=500)
print (f"average return of manual policy: {manual_reward1:.3f} avg culls {cullsteps}")

average return of do-nothing-policy: -1928.560 avg culls 0.0
average return of manual policy: -1551.621 avg culls 5.0


### Train a DDPG Agent

In [14]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        policy_state = None # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action[0][0] > 0 or action_step.action[0][1] > 0:
            cullsteps += 1
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, HerdEnv_DDPG):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>4} action: ", action_step.action, "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  return avg_return, cullsteps

In [15]:
'''Made it work to here, implement ddpg agent next.'''

import functools
from tf_agents.networks import nest_map
from tf_agents.networks import sequential
from tf_agents.keras_layers import inner_reshape
from tf_agents.networks.sequential import Sequential
from tensorflow.keras.layers import Dense
from tf_agents.agents.ddpg import ddpg_agent
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory

In [16]:
num_iterations = 20000
replay_buffer_max_length = 20000
batch_size = 64
num_eval_episodes = 100
initial_collect_steps = 2000
collect_steps_per_iteration = 5
log_interval = 500
eval_interval = 500
target_update_period = 5

In [17]:
train_env = tf_py_environment.TFPyEnvironment(HerdEnv_DDPG)
eval_env = tf_py_environment.TFPyEnvironment(HerdEnv_DDPG)

In [18]:
#kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03)

#Make critic and actor net for ddpg

dense = functools.partial(
    tf.keras.layers.Dense,
    activation=tf.keras.activations.relu,
    kernel_initializer=tf.compat.v1.variance_scaling_initializer(
        scale=1./ 3.0, mode='fan_in', distribution='uniform'))


def create_identity_layer():
    return tf.keras.layers.Lambda(lambda x: x)


def create_fc_network(layer_units):
    return sequential.Sequential([dense(num_units) for num_units in layer_units])


def create_actor_network(fc_layer_units, action_spec):
    """Create an actor network for DDPG."""
    flat_action_spec = tf.nest.flatten(action_spec)
    if len(flat_action_spec) > 1:
        raise ValueError('Only a single action tensor is supported by this network')
    flat_action_spec = flat_action_spec[0]

    fc_layers = [dense(num_units) for num_units in fc_layer_units]

    num_actions = flat_action_spec.shape.num_elements()
    action_fc_layer = tf.keras.layers.Dense(
        num_actions,
        activation=tf.keras.activations.tanh,
        kernel_initializer=tf.keras.initializers.RandomUniform(
            minval=-0.003, maxval=0.003))

    scaling_layer = tf.keras.layers.Lambda(
        lambda x: common.scale_to_spec(x, flat_action_spec))
    return sequential.Sequential(fc_layers + [action_fc_layer, scaling_layer])


def create_critic_network(obs_fc_layer_units,
                          action_fc_layer_units,
                          joint_fc_layer_units):
    """Create a critic network for DDPG."""

    def split_inputs(inputs):
        return {'observation': inputs[0], 'action': inputs[1]}

    obs_network = create_fc_network(
        obs_fc_layer_units) if obs_fc_layer_units else create_identity_layer()
    action_network = create_fc_network(
        action_fc_layer_units
        ) if action_fc_layer_units else create_identity_layer()
    joint_network = create_fc_network(
        joint_fc_layer_units) if joint_fc_layer_units else create_identity_layer()
    
    value_fc_layer = tf.keras.layers.Dense(
        1,
        activation=None,
        kernel_initializer=tf.keras.initializers.RandomUniform(
            minval=-0.003, maxval=0.003))

    return sequential.Sequential([
        tf.keras.layers.Lambda(split_inputs),
        nest_map.NestMap({
            'observation': obs_network,
            'action': action_network
        }),
        nest_map.NestFlatten(),
        tf.keras.layers.Concatenate(),
        joint_network,
        value_fc_layer,
        inner_reshape.InnerReshape([1], [])
    ])

actor_fc_layers=(400, 300)
critic_obs_fc_layers=(400,)
critic_action_fc_layers=None
critic_joint_fc_layers=(300,)

actor_net = create_actor_network(actor_fc_layers, train_env.action_spec())
critic_net = create_critic_network(critic_obs_fc_layers,
                                       critic_action_fc_layers,
                                       critic_joint_fc_layers)

In [19]:
train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

agent = ddpg_agent.DdpgAgent(
        train_env.time_step_spec(),
        train_env.action_spec(),
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=1e-4),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=1e-3),
        ou_stddev=0.3,
        ou_damping=0.15,
        target_update_tau=0.05,
        target_update_period=5,
        #dqda_clipping=dqda_clipping,
        td_errors_loss_fn=tf.compat.v1.losses.huber_loss,
        gamma=0.999,
        #reward_scale_factor=reward_scale_factor,
        #gradient_clipping=gradient_clipping,
        #debug_summaries=debug_summaries,
        #summarize_grads_and_vars=summarize_grads_and_vars,
        train_step_counter=global_step)

agent.initialize()

In [20]:
# agent.policy.trainable_variables

In [21]:
# manually initialize a reasonably good policy: kill both herds if the sum of observations is large
#W = np.array([[0, 3 ,0, 2],[0, 0, 3, 2,]])
#b = np.array([1, 0, 0, 0])
#q_net.layers[0].set_weights([W,b])
#agent.policy.trainable_variables

In [22]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)

In [23]:
agent.collect_data_spec._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

In [24]:
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)

    # Add trajectory to the replay buffer
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
    for _ in range(steps):
        collect_step(env, policy, buffer)

collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)

In [25]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

iterator = iter(dataset)
dataset

Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


<PrefetchDataset shapes: (Trajectory(
{action: (64, 2, 2),
 discount: (64, 2),
 next_step_type: (64, 2),
 observation: (64, 2, 3),
 policy_info: (),
 reward: (64, 2),
 step_type: (64, 2)}), BufferInfo(ids=(64, 2), probabilities=(64,))), types: (Trajectory(
{action: tf.float32,
 discount: tf.float32,
 next_step_type: tf.int32,
 observation: tf.float32,
 policy_info: (),
 reward: tf.float32,
 step_type: tf.int32}), BufferInfo(ids=tf.int64, probabilities=tf.float32))>

In [26]:
# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step <= 1 or step % log_interval == 0:
    print('step = {0:4>}: loss = {1:.4f}'.format(step, train_loss), end="\t")

  if step <= 1 or (step <= 100 and step % 50 == 0) or step % eval_interval == 0:
    avg_return, cullsteps = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step {0}: average return = {1:.1f} cullsteps = {2:.1f}'.format(step, avg_return.numpy().item(), cullsteps))
    returns.append(avg_return)

step = 1: loss = 15.1171	step 1: average return = -3640.0 cullsteps = 222.9
step 50: average return = -3114.6 cullsteps = 223.2
step 100: average return = -1943.0 cullsteps = 198.0
step = 500: loss = 120.2619	step 500: average return = -5372.1 cullsteps = 83.5
step = 1000: loss = 285.4420	step 1000: average return = -6065.9 cullsteps = 45.2
step = 1500: loss = 244.3414	step 1500: average return = -5312.4 cullsteps = 32.2
step = 2000: loss = 236.2220	step 2000: average return = -4989.1 cullsteps = 25.8
step = 2500: loss = 197.7295	step 2500: average return = -5250.3 cullsteps = 21.1
step = 3000: loss = 153.5066	step 3000: average return = -5262.1 cullsteps = 18.8
step = 3500: loss = 138.1592	step 3500: average return = -5417.1 cullsteps = 15.8
step = 4000: loss = 129.6016	step 4000: average return = -5710.5 cullsteps = 14.8
step = 4500: loss = 122.1865	step 4500: average return = -5257.8 cullsteps = 12.7
step = 5000: loss = 97.6707	step 5000: average return = -4903.2 cullsteps = 11.6
st

In [27]:
learned_reward, culleps = compute_avg_return(eval_env, agent.policy, num_episodes=500)
print ("reward of learned policy: ", learned_reward.numpy(), "cullsteps=", culleps)

reward of learned policy:  [-5426.526] cullsteps= 0.0


In [28]:
init_ts = eval_env.reset()

def get_action(obs):
    """ execute the learned policy network 
       obs:  one float for global time, one float for each herd - the time since last culling
    """
    _ts = TimeStep(tf.constant([0.]),
                   tf.constant([0.]),
                   tf.constant([1]),
                   tf.constant([obs]))
    # a = agent.collect_policy.action(_ts) # just to see how much is explored versus exploited
    a = agent.policy.action(_ts)
    return a.action.numpy().item()

In [29]:
agent.policy

In [30]:
# what the learned policy does on a grid of observations (5 steps per row&col)
A = [[get_action([.0, x,y])
 for y in np.arange(0.,1.,.05,np.float32)]
 for x in np.arange(0.,1.,.05,np.float32)]
A

ValueError: can only convert an array of size 1 to a Python scalar

### Play with parameters of manually designed q_network policy

In [ ]:
# W, b = agent.policy.trainable_variables
# W = W.numpy()
# b = b.numpy()
# print ("weights\n", W, "\nbias", b)

In [ ]:
# def nn(obs):
#    y = np.dot(obs, W)+b
#    return y

In [ ]:
# nn([0.5,.2])